**Data Mining Project, midterm 2021/2022**




**Authors:** Niko Dalla Noce, Alessandro Ristori, Giuseppe Lombardi

#**Task 1.1 Data Understanding**

##**Importing libraries and extracting datasets**

First we have to import the necessary libraries to work within our project.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy

We assume that the dataset is inside a zip file, so we need to extract it.

In [2]:
import zipfile
with zipfile.ZipFile("prj_data.zip", 'r') as zip_ref:
     zip_ref.extractall()

**Load the datasets**

Read the data from the three csv files, each one is assigned to a different dataframe for now.

In [3]:
df_male = pd.read_csv("male_players.csv", sep=",")  # dataframe of male players
df_female = pd.read_csv("female_players.csv", sep=",")  # dataframe of female players
df_matches = pd.read_csv("tennis_matches.csv", sep=",", index_col=0)  # matches dataframe

##**Data semantics**

###**Some informations about the dataset**

The players' dataframes, men and women, contain only the name and surname of the players (most of them are retired from the competitions and can't be found in any match record). That's why we decided to focus our work only on the match dataframe for now.

In [4]:
df_matches.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
0,2019-M020,Brisbane,Hard,32.0,A,20181231.0,300.0,105453.0,NaN,Kei Nishikori,R,178.0,JPN,95.000000,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3.0,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928.0,742618.69
1,2019-M020,Brisbane,Hard,32.0,A,20181231.0,299.0,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,104542.0,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3.0,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928.0,742618.69
2,2019-M020,Brisbane,Hard,32.0,A,20181231.0,298.0,105453.0,NaN,Kei Nishikori,R,178.0,JPN,29.004791,104871.0,NaN,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3.0,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928.0,742618.69
3,2019-M020,Brisbane,Hard,32.0,A,20181231.0,297.0,104542.0,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282.0,NaN,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3.0,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928.0,742618.69
4,2019-M020,Brisbane,Hard,32.0,A,20181231.0,296.0,106421.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,105683.0,NaN,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3.0,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928.0,742618.69


We found that the last records of the matches dataset (the Taipei tournament) are missing a lot of values.

In [5]:
df_matches.tail()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
186123,NaN,NaN,Hard,32.0,NaN,20171113.0,250.0,NaN,NaN,NaN,NaN,NaN,NaN,26.206708,206075.0,NaN,NaN,NaN,NaN,NaN,26.90486,3-6 6-2 6-3,NaN,NaN,NaN,NaN,1.0,NaN,49.0,38.0,16.0,NaN,7.0,NaN,1.0,3.0,93.0,NaN,NaN,NaN,NaN,7.0,11.0,NaN,322.0,280.0,183.0,NaN,NaN
186124,NaN,Taipei,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Veronika Kudermetova,NaN,175.0,NaN,NaN,214562.0,WC,Ingrid Neel,NaN,NaN,USA,NaN,7-6(3) 6-2,NaN,NaN,71.0,10.0,8.0,63.0,NaN,32.0,NaN,10.0,2.0,5.0,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,255.0,217.0,704.0,NaN,NaN,NaN
186125,NaN,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,Dalila Jakupovic,NaN,NaN,NaN,26.642026,201674.0,NaN,NaN,R,NaN,UZB,NaN,NaN,3.0,NaN,70.0,NaN,2.0,55.0,NaN,NaN,NaN,NaN,6.0,8.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
186126,NaN,NaN,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Vitalia Diatchenko,NaN,170.0,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-6 6-2 6-4,3.0,NaN,115.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,0.0,75.0,NaN,NaN,13.0,NaN,3.0,NaN,176.0,NaN,NaN,NaN,3535.0,NaN
186127,NaN,Taipei,Hard,32.0,C,20171113.0,NaN,NaN,NaN,Priscilla Hon,NaN,NaN,AUS,19.512663,NaN,NaN,NaN,NaN,NaN,USA,NaN,6-4 6-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,NaN,NaN,43.0,25.0,9.0,10.0,4.0,8.0,NaN,NaN,181.0,322.0,3535.0,638954.72


In [6]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186128 entries, 0 to 186127
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          186073 non-null  object 
 1   tourney_name        186103 non-null  object 
 2   surface             185940 non-null  object 
 3   draw_size           186099 non-null  float64
 4   tourney_level       186099 non-null  object 
 5   tourney_date        186100 non-null  float64
 6   match_num           186101 non-null  float64
 7   winner_id           186073 non-null  float64
 8   winner_entry        25827 non-null   object 
 9   winner_name         186101 non-null  object 
 10  winner_hand         186082 non-null  object 
 11  winner_ht           49341 non-null   float64
 12  winner_ioc          186099 non-null  object 
 13  winner_age          183275 non-null  float64
 14  loser_id            186100 non-null  float64
 15  loser_entry         44154 non-null

###**Type casting**

Most of the attributes have a wrong type, we need to fix it.

In [7]:
df_matches.dtypes

tourney_id             object
tourney_name           object
surface                object
draw_size             float64
tourney_level          object
tourney_date          float64
match_num             float64
winner_id             float64
winner_entry           object
winner_name            object
winner_hand            object
winner_ht             float64
winner_ioc             object
winner_age            float64
loser_id              float64
loser_entry            object
loser_name             object
loser_hand             object
loser_ht              float64
loser_ioc              object
loser_age             float64
score                  object
best_of               float64
round                  object
minutes               float64
w_ace                 float64
w_df                  float64
w_svpt                float64
w_1stIn               float64
w_1stWon              float64
w_2ndWon              float64
w_SvGms               float64
w_bpSaved             float64
w_bpFaced 

In [8]:
df_matches['tourney_date']= df_matches['tourney_date'].astype(pd.Int32Dtype(), errors='ignore').astype(pd.StringDtype())
df_matches['tourney_date'] = pd.to_datetime(df_matches['tourney_date'], errors='coerce', format='%Y%m%d')
df_matches.iloc[:, [3, 6, 7, 11, 14, 18, 22]] = df_matches.iloc[:, [3, 6, 7, 11, 14, 18, 22]].astype("Int64", errors="ignore")
df_matches.iloc[:, 24:48] = df_matches.iloc[:, 24:48].astype("Int64", errors="ignore")
df_matches.dtypes

tourney_id                    object
tourney_name                  object
surface                       object
draw_size                      Int64
tourney_level                 object
tourney_date          datetime64[ns]
match_num                      Int64
winner_id                      Int64
winner_entry                  object
winner_name                   object
winner_hand                   object
winner_ht                      Int64
winner_ioc                    object
winner_age                   float64
loser_id                       Int64
loser_entry                   object
loser_name                    object
loser_hand                    object
loser_ht                       Int64
loser_ioc                     object
loser_age                    float64
score                         object
best_of                        Int64
round                         object
minutes                        Int64
w_ace                          Int64
w_df                           Int64
w

###**Dropping Duplicates**

Since there shouldn't be any duplitcated matches, we need to find and drop them.

In [9]:
ndupl = df_matches.duplicated()
ndupl = df_matches[ndupl]
print("Number of duplicated matches: {0}".format(len(ndupl)))
ndupl

Number of duplicated matches: 309


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
12066,2017-1059,Taipei,Hard,32,C,2017-11-13,243,214562,WC,Ingrid Neel,U,<NA>,USA,19.411362,203475,NaN,Carol Zhao,R,<NA>,CAN,22.401095,6-7(7) 7-6(6) 6-2,3,Q1,154,7,10,124,84,53,14,16,7,13,9,17,151,90,54,21,16,14,22,704,27,150,390,3535,638954.72
12067,2017-1059,Taipei,Hard,32,C,2017-11-13,244,206368,NaN,Veronika Kudermetova,R,175,RUS,20.555784,203576,NaN,Kai Lin Zhang,R,<NA>,CHN,27.791923,7-6(2) 7-5,3,Q1,104,4,6,87,54,36,13,12,5,10,0,8,82,50,23,15,12,3,9,255,217,208,274,3535,638954.72
12068,2017-1059,Taipei,Hard,32,C,2017-11-13,245,203435,NaN,Dalila Jakupovic,R,<NA>,SLO,26.642026,211874,NaN,Victoria Rodriguez,U,<NA>,MEX,22.562628,6-2 6-3,3,Q1,69,0,2,46,29,19,8,9,2,5,0,3,60,42,19,6,8,5,11,242,228,262,204,3535,638954.72
12069,2017-1059,Taipei,Hard,32,C,2017-11-13,246,201674,NaN,Sabina Sharipova,R,<NA>,UZB,23.192334,214541,NaN,Fanny Stollar,R,172,HUN,19.003422,6-1 7-6(2),3,Q1,80,2,4,71,43,30,13,10,3,6,4,7,80,45,27,10,9,7,12,196,299,238,231,3535,638954.72
12070,2017-1059,Taipei,Hard,32,C,2017-11-13,247,211411,NaN,Nina Stojanovic,R,<NA>,SRB,21.289528,204291,NaN,Na Lae Han,L,<NA>,KOR,25.355236,7-6(3) 6-3,3,Q1,91,5,3,79,49,34,11,11,4,8,0,4,68,41,23,9,10,2,7,230,243,250,221,3535,638954.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177376,2020-1103,Indian Wells 125K,Hard,64,C,2020-03-02,221,203398,NaN,Danielle Lao,R,<NA>,USA,28.763860,213949,WC,Hanna Chang,U,<NA>,USA,22.015058,6-1 5-7 6-2,3,Q1,170,0,3,119,86,54,16,14,12,15,3,7,84,52,30,14,13,3,9,198,296,295,184,4638,1164466.18
177377,2020-1103,Indian Wells 125K,Hard,64,C,2020-03-02,222,201534,NaN,Asia Muhammed,R,<NA>,USA,28.911704,215064,WC,Maria Mateas,U,<NA>,USA,20.616016,7-5 2-6 6-3,3,Q1,157,3,4,100,60,32,14,14,13,22,0,6,100,75,39,6,15,10,20,194,304,413,103,4638,1164466.18
177378,2020-1103,Indian Wells 125K,Hard,64,C,2020-03-02,221,203398,NaN,Danielle Lao,R,<NA>,USA,28.763860,213949,WC,Hanna Chang,U,<NA>,USA,22.015058,6-1 5-7 6-2,3,Q1,170,0,3,119,86,54,16,14,12,15,3,7,84,52,30,14,13,3,9,198,296,295,184,4638,1164466.18
177379,2020-1103,Indian Wells 125K,Hard,64,C,2020-03-02,222,201534,NaN,Asia Muhammed,R,<NA>,USA,28.911704,215064,WC,Maria Mateas,U,<NA>,USA,20.616016,7-5 2-6 6-3,3,Q1,157,3,4,100,60,32,14,14,13,22,0,6,100,75,39,6,15,10,20,194,304,413,103,4638,1164466.18


We can drop all the duplicates now.

In [10]:
df_matches = df_matches.drop_duplicates()

###**Dealing with Missing Values**

We have to deal with missing values for each attribute.

In [11]:
print("NULL FIELDS:")
print(df_matches.isnull().any())

NULL FIELDS:
tourney_id            True
tourney_name          True
surface               True
draw_size             True
tourney_level         True
tourney_date          True
match_num             True
winner_id             True
winner_entry          True
winner_name           True
winner_hand           True
winner_ht             True
winner_ioc            True
winner_age            True
loser_id              True
loser_entry           True
loser_name            True
loser_hand            True
loser_ht              True
loser_ioc             True
loser_age             True
score                 True
best_of               True
round                 True
minutes               True
w_ace                 True
w_df                  True
w_svpt                True
w_1stIn               True
w_1stWon              True
w_2ndWon              True
w_SvGms               True
w_bpSaved             True
w_bpFaced             True
l_ace                 True
l_df                  True
l_svpt         

**Fixing tourney_id**

The attribute tourney_id has missing values in corrispondece with only the Taipei tournament (which doesn't have many more values), moreover we found that this event is already in the dataset with better filled records, we decided to drop it from the dataframe.

In [12]:
df_matches = df_matches.dropna(subset=["tourney_id"])
df_matches.isnull().any()

tourney_id            False
tourney_name          False
surface                True
draw_size             False
tourney_level         False
tourney_date          False
match_num             False
winner_id             False
winner_entry           True
winner_name           False
winner_hand            True
winner_ht              True
winner_ioc            False
winner_age             True
loser_id              False
loser_entry            True
loser_name            False
loser_hand             True
loser_ht               True
loser_ioc             False
loser_age              True
score                  True
best_of               False
round                 False
minutes                True
w_ace                  True
w_df                   True
w_svpt                 True
w_1stIn                True
w_1stWon               True
w_2ndWon               True
w_SvGms                True
w_bpSaved              True
w_bpFaced              True
l_ace                  True
l_df                

Come possiamo notare, eliminando il torneo di Taipei molti attributi sono adesso senza valori nulli, questo ci rende l'analisi ancora più semplice. Inoltre, tale torneo è già presente nel dataset e con valori corretti, perciò la sua eliminazione non influisce negativamente sul nostro lavoro.

**Fixing surface**

We try to solve the null values in the "surface" feature, first we count how many tournaments and matches have no surfaces for pure statistical reasons. If a tournament is missing the surface we can retrieve it if at least one match of the same tournament has it.

In [13]:
# tournaments with no surface
tourney_no_surface = df_matches[pd.isnull(df_matches['surface'])]['tourney_id'].unique()

# matches with no surface
matches_no_surface = df_matches[df_matches['tourney_id'].isin(tourney_no_surface)][['tourney_id', 'surface']]

print("Number of tournaments without a surface: {0}".format(len(tourney_no_surface)))
print("Number of matches without a surface: {0}".format(len(matches_no_surface)))
print("Number of tournaments for which we can retrieve the surface: {0}".
      format(len(matches_no_surface[matches_no_surface['surface'].notnull()])))

Number of tournaments without a surface: 62
Number of matches without a surface: 162
Number of tournaments for which we can retrieve the surface: 0


Unluckily there are no tournaments for which we can retrieve the surface, so we chose to sample such values from the distribution of the surfaces.

In [14]:
# all possible values of "surface" attribute normalized
surface_distribution = df_matches["surface"].value_counts(normalize=True)
print("Distribution of each surface:\n{0}\n".format(surface_distribution))
print("Distribution of each surface cumulative:\n{0}\n".format(surface_distribution.cumsum()))

# sample surfaces and assing them to each tournament
for tourney in tourney_no_surface:
  sampled_surface = surface_distribution.sample(weights=surface_distribution, axis=0).index[0]
  print("Tournament: {0}, Surface sampled: {1}".format(tourney, sampled_surface))
  surface_mask = df_matches["tourney_id"] == tourney
  df_matches.loc[surface_mask, "surface"] = df_matches.loc[surface_mask, "surface"].fillna(sampled_surface)

Distribution of each surface:
Hard      0.511503
Clay      0.436488
Grass     0.035560
Carpet    0.016449
Name: surface, dtype: float64

Distribution of each surface cumulative:
Hard      0.511503
Clay      0.947991
Grass     0.983551
Carpet    1.000000
Name: surface, dtype: float64

Tournament: 2017-W-FC-2017-G2-AM-A-M-ECU-CUB-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-A-M-ECU-URU-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-A-M-URU-CUB-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-B-M-DOM-BAR-01, Surface sampled: Clay
Tournament: 2017-W-FC-2017-G2-AM-B-M-GUA-BAR-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-B-M-GUA-DOM-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-C-M-PUR-HON-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-C-M-PUR-TTO-01, Surface sampled: Clay
Tournament: 2017-W-FC-2017-G2-AM-C-M-TTO-HON-01, Surface sampled: Hard
Tournament: 2017-W-FC-2017-G2-AM-D-M-BAH-PAN-01, Surface sampled: Hard
Tourn

In [15]:
df_matches["surface"].isnull().any()

False

**Fixing winner_hand and loser_hand**

We have to manage the missing values for "winner_hand" and "loser_hand", for doing that we need to understand if the preferred hand is missing only in some records or it never appear for a certain player. Moreover, we have to verify is the missing hand for the winners is still null when those players appear inside the "loser_name" attribute (and viceversa).

For what concerns the preferred hand by both winners and losers, its possible values are "L", "R" or "U". Many records have null values and so it could mean that we can assume that those values can be replace with "U".

In [17]:
df_matches['winner_hand'].fillna('U', inplace=True)
df_matches['loser_hand'].fillna('U', inplace=True)

Let's see if there are players with preferred hand "U" but, actually, their hand is already defined "L" or "R" inside one of the record of the dataset, we can modify all the matches they belong to.

In [18]:
def fill_attribute_with_actual_values(attribute: str):
  winner_attribute = "winner_" + attribute
  loser_attribute = "loser_" + attribute

  # player that have won or lost at least one match with unknown attribute
  winner_unk_attribute = df_matches[df_matches[winner_attribute] == 'U']['winner_name'].unique()
  loser_unk_attribute = df_matches[df_matches[loser_attribute] == 'U']['loser_name'].unique()
  players_unk_attribute = set(np.concatenate((winner_unk_attribute, loser_unk_attribute)))
  print("Players with unknown {0}: {1}\n".format(attribute, len(players_unk_attribute)))

  # all the winners that appear in the players_unk_attribute set
  w = df_matches[df_matches['winner_name'].isin(players_unk_attribute)][['winner_name', winner_attribute]]
  w = w[w[winner_attribute] != 'U'].drop_duplicates()

  # all the winners that appear in the players_unk_attribute set
  l = df_matches[df_matches['loser_name'].isin(players_unk_attribute)][['loser_name', loser_attribute]]
  l = l[l['loser_hand'] != 'U'].drop_duplicates()

  l.rename(columns={'loser_name': 'player', loser_attribute: attribute}, inplace=True)
  w.rename(columns={'winner_name': 'player', winner_attribute: attribute}, inplace=True)

  # concatenate the winners and losers
  winner_loser_attribute = pd.concat([l, w]).drop_duplicates()
  print("Players that belong to records in which they have no {0} defined:\n{1}\n".format(attribute, winner_loser_attribute))

  for _, x in winner_loser_attribute.iterrows():
    attribute_actual_value = x[attribute]
    df_matches.loc[df_matches['winner_name'] == x['player'], winner_attribute] = attribute_actual_value
    df_matches.loc[df_matches['loser_name'] == x['player'], loser_attribute] = attribute_actual_value

In [19]:
fill_attribute_with_actual_values("hand")

Players with unknown hand: 6307

Players that belong to records in which they have no hand defined:
                      player hand
468       Ekaterina Makarova    L
1809            Qinwen Zheng    R
54654   Hei Ching Claudia Ng    R
64643               Eva Shaw    R
99997           Amina Anshba    L
100012       Weronika Baszak    R
139277    Anastasia Kulikova    L
164256         Vanessa Ersoz    R



Now we can sample a preferred hand for each player with that unknown value.

In [46]:
w_hand = df_matches[["winner_name", "winner_hand"]].rename(columns={"winner_name": "name", "winner_hand": "hand"})
l_hand = df_matches[["loser_name", "loser_hand"]].rename(columns={"loser_name": "name", "loser_hand": "hand"})
players_hand = w_hand.append(l_hand).drop_duplicates()
hand_distribution = players_hand[players_hand["hand"]!="U"]["hand"].value_counts(normalize=True)
print("Distribution of each preferred hand:\n{0}\n".format(hand_distribution))
print("Distribution of each preferred hand cumulative:\n{0}\n".format(hand_distribution.cumsum()))

Distribution of each preferred hand:
R    0.893824
L    0.106176
Name: hand, dtype: float64

Distribution of each preferred hand cumulative:
R    0.893824
L    1.000000
Name: hand, dtype: float64



In [58]:
players_unk_hand = players_hand[players_hand["hand"]=="U"]["name"]


for player in players_unk_hand:
  sampled_hand = hand_distribution.sample(weights=hand_distribution, axis=0).index[0]
  df_matches.loc[df_matches['winner_name'] == player, "winner_hand"] = sampled_hand
  df_matches.loc[df_matches['loser_name'] == player, "loser_hand"] = sampled_hand

In [69]:
print("Winners with unknown hand: {0}".format(len(df_matches[df_matches["winner_hand"] == "U"])))
print("Losers with unknown hand: {0}".format(len(df_matches[df_matches["loser_hand"] == "U"])))

Winners with unknown hand: 0
Losers with unknown hand: 0


In [59]:
print(df_matches["winner_hand"].isnull().any())
print(df_matches["loser_hand"].isnull().any())

False
False


**Fixing winner_ht and loser_ht**

Le features winner_ht e loser_ht presentano anch'esse dei valori nulli.

In [ ]:
winner_nan_ht = df_matches[pd.isna(df_matches['winner_ht'])]['winner_name'].unique()
w = df_matches[df_matches['winner_name'].isin(winner_nan_ht)][['winner_name', 'winner_ht']]
w = w[w['winner_ht'].notnull()].drop_duplicates()

l = df_matches[df_matches['loser_name'].isin(winner_nan_ht)][['loser_name', 'loser_ht']]
l = l[l['loser_ht'].notnull()].drop_duplicates()

l.rename(columns={'loser_name': 'player', 'loser_ht': 'ht'}, inplace=True)
w.rename(columns={'winner_name': 'player', 'winner_ht': 'ht'}, inplace=True)

tot = pd.concat([l, w]).drop_duplicates()

for i, x in tot.iterrows():
    ht = x['ht']
    df_matches.loc[df_matches['winner_name'] == x['player'], 'winner_ht'] = ht

In [ ]:
loser_nan_ht = df_matches[pd.isna(df_matches['loser_ht'])]['loser_name'].unique()
l = df_matches[df_matches['loser_name'].isin(loser_nan_ht)][['loser_name', 'loser_ht']]
l = l[l['loser_ht'].notnull()].drop_duplicates()

w = df_matches[df_matches['winner_name'].isin(loser_nan_ht)][['winner_name', 'winner_ht']]
w = w[w['winner_ht'].notnull()].drop_duplicates()

l.rename(columns={'loser_name': 'player', 'loser_ht': 'ht'}, inplace=True)
w.rename(columns={'winner_name': 'player', 'winner_ht': 'ht'}, inplace=True)

tot = pd.concat([l, w]).drop_duplicates()

for i, x in tot.iterrows():
    ht = x['ht']
    df_matches.loc[df_matches['loser_name'] == x['player'], 'loser_ht'] = ht


In [ ]:
print(df_matches.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185764 entries, 0 to 186072
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   tourney_id          185764 non-null  object        
 1   tourney_name        185764 non-null  object        
 2   surface             185764 non-null  object        
 3   draw_size           185764 non-null  Int64         
 4   tourney_level       185764 non-null  object        
 5   tourney_date        185764 non-null  datetime64[ns]
 6   match_num           185764 non-null  Int64         
 7   winner_id           185764 non-null  Int64         
 8   winner_entry        25806 non-null   object        
 9   winner_name         185764 non-null  object        
 10  winner_hand         185764 non-null  object        
 11  winner_ht           51402 non-null   Int64         
 12  winner_ioc          185764 non-null  object        
 13  winner_age          182936 no

###**Outlier Detection**

Calcolo il primo quartile, il terzo quartile delle features e gli interquartili

In [ ]:
Q1 = df_matches.quantile(0.25)
Q3 = df_matches.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Definisco i Lower Bound e gli Upper Bound per ogni features

In [ ]:
L = Q1 - 1.5 * IQR
U = Q3 + 1.5 * IQR

Attributo altezza del winner ('winner_ht')

In [ ]:
df_matches.boxplot(column=["winner_ht", 'loser_ht']);

C'è qualche valore molto basso

In [ ]:
df_matches[['winner_ht', 'loser_ht']].plot.hist(stacked=True, bins=100);

Cerco players troppo alti o troppo bassi

In [ ]:
df_matches[(df_matches['winner_ht'] < (Q1['winner_ht'] - 3 * IQR['winner_ht'])) |(df_matches['winner_ht'] > (Q3['winner_ht'] + 3 * IQR['winner_ht']))]['winner_ht']

In [ ]:
df_matches[(df_matches['winner_ht'] < (Q1['winner_ht'] - 3 * IQR['winner_ht'])) |(df_matches['winner_ht'] > (Q3['winner_ht'] + 3 * IQR['winner_ht']))]['winner_name'].unique()

Sostituisco il valore troppo basso con il Lower Bound L

---



In [ ]:
df_matches.loc[(df_matches['winner_ht'] < (Q1['winner_ht'] - 3 * IQR['winner_ht'])),'winner_ht'] = int(max(L['winner_ht'], L['loser_ht']))

In [ ]:
df_matches.loc[(df_matches['loser_ht'] < (Q1['loser_ht'] - 3 * IQR['loser_ht'])),'loser_ht'] = int(max(L['winner_ht'], L['loser_ht']))

Ci sono ancora giocatori molto alti (ma lo sono realmente)

In [ ]:
df_matches[(df_matches['winner_ht'] > (U['winner_ht'])]['winner_name'].unique()

C'è ancora un player molto basso (la sua vera altezza è un'altra). Cosa fare?

In [ ]:
df_matches[(df_matches['winner_ht'] < (Q3['winner_ht'] - 3 * IQR['winner_ht']))]['winner_name'].unique()

In [ ]:
df_matches.loc[df_matches['winner_name'] == 'Ilija Vucic','winner_ht']

In [ ]:
df_matches.boxplot(column=["winner_age", "loser_age"]);

In [ ]:
df_matches[(df_matches['winner_age'] < (Q1['winner_age'] - 3 * IQR['winner_age'])) |(df_matches['winner_age'] > (Q3['winner_age'] + 3 * IQR['winner_age']))]['winner_age']

##Correlations

In [ ]:
categorical_features = ['tourney_id','tourney_name','surface','tourney_level', 'tourney_date','match_num', 'winner_id', 'winner_entry', 'winner_name', 'winner_hand', 'winner_ioc', 'loser_id', 'loser_entry', 'loser_name', 'loser_hand', 'loser_ioc', 'score', 'round']
numerical_features = ['draw_size', 'winner_ht',	'winner_age', 'loser_ht',	'loser_age', 'best_of',	'minutes', 'w_ace',	'w_df',	'w_svpt',	'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced',	'l_ace', 'l_df', 'l_svpt', 'l_1stIn',	'l_1stWon',	'l_2ndWon',	'l_SvGms', 'l_bpSaved',	'l_bpFaced', 'winner_rank',	'winner_rank_points',	'loser_rank',	'loser_rank_points', 'tourney_spectators', 'tourney_revenue']

In [ ]:
df_matches_numerical=df_matches[numerical_features]

In [ ]:
df_matches_numerical.corr()

Searching highly-corraleted features

In [ ]:
features = df_matches_numerical.columns
correl_matrix = np.array(df_matches_numerical.corr())
correl_attrib = []
n_row, n_col = correl_matrix.shape[0], correl_matrix.shape[1]

CORREL_THRESHOLD = 0.9

for i in range(n_row):
    for j in range(i + 1, n_col):
        if correl_matrix[i, j] > CORREL_THRESHOLD or correl_matrix[i, j] < -CORREL_THRESHOLD:
            correl_attrib.append((features[i], features[j]))

correl_attrib

**"Categorical" features**

In [ ]:
df_matches_categorical = df_matches[categorical_features]

In [ ]:
df_matches_categorical.head()

#**Questa parte è data preparation** Now let's create a new dataset for the players

In [ ]:
players_w = df_matches[["winner_name"]].rename(columns={"winner_name":"Name"})  # player names extracted from the winners
players_l = df_matches[["loser_name"]].rename(columns={"loser_name":"Name"})  # player names extracted from the losers

# first we append the losers list to the winners one
df_players = players_w.append(players_l)

# the redundant records are dropped and a new dataframe is built from the list of players
df_players = df_players.drop_duplicates()
print("{0}\n".format(df_players.info()))
print(df_players.isnull().any())

There's only one record with no value, let's drop it.

In [ ]:
df_players = df_players.dropna()
df_players.isnull().any()

The new dataframe has only one attribute, as the head() method shows.

In [ ]:
df_players.head()

Now, we need to know if a player is male or female, that's when df_male and df_female come to help us.

In [ ]:
def players_from_sex(df_players_sex, df_players_matches):
  """
  Obtain the players of a specific sex given a dataset of that sex
  :param df_players_sex: players dataframe (M or F) from where we obtain the sex
  :param df_players_matches: players dataframe built from the matches
  :return players: list of players (M or F)
  :return index: players indexes (M or F)
  """
  players_name = list(df_players_sex["name"])
  players_surname = list(df_players_sex["surname"])
    
  #lista di players nome e cognome da df_players
  players = list()
  for n, s in zip(players_name, players_surname):
    players.append("{0} {1}".format(n, s))

  players = df_players_matches[df_players_matches["Name"].isin(players)]

  players_indexes = players.index

  return list(players["Name"]), players_indexes

In [ ]:
[players_m, i_male_found] = players_from_sex(df_male, df_players)
[players_f, i_female_found] = players_from_sex(df_female, df_players)

# players for which we have found the sex
players_sex_found = players_m+players_f

# players that belong to both datasets
players_both_sex = list(set(players_m) & set(players_f))
print("Players found in both datasets (male and female):\n{0}\n".format(players_both_sex))

# players with no sex
players_sex_not_found = df_players[~df_players["Name"].isin(players_sex_found)]

# some stats about what we found
print("Number of male players: {0}".format(len(players_m)))
print("Number of female players: {0}".format(len(players_f)))
print("Number of players: {0}".format(len(df_players)))
print("Number of players with no sex known: {0}\n".format(len(players_sex_not_found)))
players_sex_not_found

All of the players with no known sex are in one of the datasets (male or female) but their name is written differently (e.g.: uppercase in matches but undercase in males), so we need to assign them a sex by looking at the tournaments they took part.

In [ ]:
# let's insert a new attribute (sex) and assign the right values
df_players.loc[i_male_found, 'Sex'] = 'M'
df_players.loc[i_female_found, 'Sex'] = 'F'
df_players.info()

To obtain the sex of those players with no known sex we can take a look at the tournaments' levels they took part in, since there's only one type that is shared by both sexes.

In [ ]:
# tournament levels for each sex
male_t = {'M', 'G', 'C', 'A', 'S', 'F'}
female_t = {'P', 'PM', 'I', 'T1', '15', '25', '60', '100', '80', '10', '50', '75'}

players_sex_not_found_i = players_sex_not_found.index
for player, i in zip(players_sex_not_found["Name"], players_sex_not_found_i):
  player_matches_info = df_matches[df_matches['loser_name']==player]
  player_t_levels = set(player_matches_info["tourney_level"].dropna())
  if len(player_t_levels.intersection(male_t)) > 0:
    sex = "M"
    players_m.insert(i, player)
  else:
    if len(player_t_levels.intersection(female_t)) > 0:
      sex = "F"
      players_f.insert(i, player)
    else:
      sex = "Not known"
  
  df_players.loc[i, "Sex"] = sex
  print("{0} {1}".format(player, sex))

Now there are no more missing values for the sex attribute and each player is either M or F.

In [ ]:
df_players.info()

We still need to work on those players who appear in both the male and female datasets.

In [ ]:
players_both_sex = df_players[df_players["Name"].isin(players_both_sex)]
players_both_sex_i = players_both_sex.index
players_both_sex

**Non funziona**

In [ ]:
players_both_sex_i = players_both_sex.index
for player, i in zip(players_both_sex["Name"], players_both_sex_i):
  player_matches_info = df_matches[df_matches['loser_name']==player]
  player_t_levels = set(player_matches_info["tourney_level"].dropna())
  if len(player_t_levels.intersection(male_t)) > 0 & len(player_t_levels.intersection(female_t)) == 0:
    sex = "M"
    players_m.remove(player)
  else:
    if len(player_t_levels.intersection(female_t)) > 0 & len(player_t_levels.intersection(male_t)) == 0:
      sex = "F"
      players_f.remove(player)
    else:
      sex = "Not known"
  
  df_players.loc[i, "Sex"] = sex
  print("{0} {1}".format(player, sex))

Every player has just one sex after our work.



In [ ]:
print(len(players_f))
print(len(players_m))
print(len(df_players))

#**Cambiare tutto da qui in poi**

Let's take a look to the players dataframes by calling the head() method, which shows the first five lines of the dataframes.

In [ ]:
df_male.head()

In [ ]:
df_female.head()

Let's now use the info() method to obtain information on the two player datasets.

In [ ]:
print("Male players dataframe info")
df_male.info()
print("\nFemale players dataframe info")
df_female.info()

As we can see from the previous infos, there are null values both in the male and female dataset.

In [ ]:
print("Male players dataframe null values:\n{0}".format(df_male.isnull().any()))
print("\nFemale players dataframe null values:\n{0}".format(df_female.isnull().any()))

We need to drop such elements with null values.

In [ ]:
df_male = df_male.dropna()
df_female = df_female.dropna()
print("Male players dataframe info:")
df_male.info()
print("\nFemale players dataframe info:")
df_female.info()

There could be duplicates in the same dataset, but we know that there's a possibility that two players could share the same name and surname, so we won't change anything.

But, there are elements where the values are missing, or incomplete, but not null (look at df_female.head()), we need to remove such elements.

In [ ]:
elements_not_to_keep = ["Unknown", "??"]
df_male = df_male[df_male["name"].str.len()>1]
df_male = df_male[df_male["surname"].str.len()>1]
df_male = df_male[~df_male["name"].isin(elements_not_to_keep)]
df_male = df_male[~df_male["surname"].isin(elements_not_to_keep)]
df_male.info()
# df_male[df_male["name"].isin(["??"])]
# df_male.where(df_male.name.str.len()==1)
# Ci sono altri elementi che non sono compatibili, andrebbero visti a mano